In [45]:
from pathlib import Path
import mat73
import mne
import numpy as np
import mne.io as mnio

def add_labels_to_stim(raw, n_trials=4):
    events = mne.find_events(raw, stim_channel='stim', output='step')

    on_set_mask = events[:,2] == 1 # stim channel goes from 0 to 1
    on_set_events = events[on_set_mask, 0]

    off_set_mask = events[:,2] == 0  # stim channel goes from 1 to 0
    off_set_events = events[off_set_mask,0]

    raw_copy = raw.copy() # do not effect the original raw signal
    data = raw_copy.get_data() # directly access the underlying numpy data!

    # loop trough events and encode the label as a stim channel event_id
    for i, (on_set, off_set) in enumerate(zip(on_set_events, off_set_events)):
        trial_label = (i % n_trials) +1 # just a trick to avoid looping i in range(1,5)
        data[8, on_set:off_set] = trial_label # modify the stim channel (index 8)

    return raw_copy

def load_fif_data(path):
    return mne.io.read_raw_fif(path)


def get_trials(patient_session_path, load_func=None):

    SFREQ = 256
    CH_NAMES = ['P1','P3','P5','P7','P10','O2','O7','O8']

    raw = load_func(patient_session_path)
    raw2 = add_labels_to_stim(raw)

    modified_stim, times = raw2[8] # 8 index of stim channel

    stim_events = {'9Hz': 1, '10Hz': 2, '12Hz': 3, '15Hz': 4}
    events = mne.find_events(raw2, stim_channel='stim')
    TRIAL_DURATION = 7.35
    epochs = mne.Epochs(raw, events, event_id=stim_events, tmin=-0.005, tmax=TRIAL_DURATION, picks=['eeg'], baseline=None) # each trial is about 7.35 s from onset stimulus 

    return epochs.get_data()

def load_patient_trials(subject=1, base_path="."):
    subj_session1 = Path(base_path) / Path(f'../subject_{subject}_fvep_led_training_1.mat')
    #subj_session2 = Path(base_path) / Path(f'../subject_{subject}_fvep_led_training_2.mat')

    
    #session_data_2 = get_trials(subj_session2, load_fif_data)
    labels = np.array([0,1,2,3] * 5)
    
    return session_data_1, labels, session_data_2, labels




In [52]:
subject_1_session_1_raw = get_trials("Risultati/raw.fif", load_fif_data)
subject_1_session_1_CAR = get_trials("Risultati/filt_CAR.fif", load_fif_data)
subject_1_session_1_ICA_CAR = get_trials("Risultati/filt_ica_car.fif", load_fif_data)
subject_1_session_1_ICA = get_trials("Risultati/filt_ICA.fif", load_fif_data)

np.save("trials/subject_1_session_1_raw.npy", subject_1_session_1_raw)
np.save("trials/subject_1_session_1_CAR.npy", subject_1_session_1_CAR )
np.save("trials/subject_1_session_1_ICA_CAR.npy", subject_1_session_1_ICA_CAR)
np.save("trials/subject_1_session_1_ICA.npy", subject_1_session_1_ICA)



Opening raw data file Risultati/raw.fif...
Isotrak not found
    Range : 0 ... 57727 =      0.000 ...   225.496 secs
Ready.
40 events found
Event IDs: [0 1 2 3 4]
20 events found
Event IDs: [1 2 3 4]
Not setting metadata
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Loading data for 20 events and 1884 original time points ...
0 bad epochs dropped
Opening raw data file Risultati/filt_CAR.fif...
Isotrak not found
    Range : 0 ... 57727 =      0.000 ...   225.496 secs
Ready.
40 events found
Event IDs: [0 1 2 3 4]
20 events found
Event IDs: [1 2 3 4]
Not setting metadata
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Loading data for 20 events and 1884 original time points ...
0 bad epochs dropped
Opening raw data file Risultati/filt_ica_car.fif...
Isotrak not found
    Range : 0 ... 57727 =      0.000 ...   225.496 secs
Ready.
40 events found
Event IDs: [0 1 2 3 4]
20 ev

/tmp/ipykernel_2718/639495823.py:27: RuntimeWarning: This filename (Risultati/filt_CAR.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  return mne.io.read_raw_fif(path)
/tmp/ipykernel_2718/639495823.py:27: RuntimeWarning: This filename (Risultati/filt_ica_car.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  return mne.io.read_raw_fif(path)
/tmp/ipykernel_2718/639495823.py:27: RuntimeWarning: This filename (Risultati/filt_ICA.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.g

40 events found
Event IDs: [0 1 2 3 4]
20 events found
Event IDs: [1 2 3 4]
Not setting metadata
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Loading data for 20 events and 1884 original time points ...
0 bad epochs dropped
